In [ ]:
import kaggle
!kaggle datasets download -d thedevastator/unlock-profits-with-e-commerce-sales-data

In [ ]:
import zipfile
with zipfile.ZipFile('unlock-profits-with-e-commerce-sales-data.zip','r') as zip_ref:
    zip_ref.extractall('Dataset_Folder')

In [45]:
#Know your data(Dimensions)
import pandas as pd
df = pd.read_csv('Dataset_Folder/Amazon Sale Report.csv',low_memory=False)
print("Rows and Coloumns:",df.shape)

Rows and Coloumns: (128975, 24)


In [46]:
#Dropping of the unuse data
#Rename the columns 
#Remove the duplicate rows
df= df.drop('Unnamed: 22',axis=1)
df.columns=df.columns.str.strip().str.replace(r'[ -]','_',regex=True).str.lower()
df=df.drop_duplicates()
df=df.rename(columns= {'date':'order_date','qty':'quantity','amount':'order_amount'})
df.columns

Index(['index', 'order_id', 'order_date', 'status', 'fulfilment',
       'sales_channel', 'ship_service_level', 'style', 'sku', 'category',
       'size', 'asin', 'courier_status', 'quantity', 'currency',
       'order_amount', 'ship_city', 'ship_state', 'ship_postal_code',
       'ship_country', 'promotion_ids', 'b2b', 'fulfilled_by'],
      dtype='object')

In [67]:
# Month mapping
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

# Convert proper dates directly
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce', dayfirst=True)

# Fill month names and map them to numeric equivalents
month_mask = df['order_date'].isna() & df['order_date'].astype(str).isin(month_map)
df.loc[month_mask, 'order_date'] = df.loc[month_mask, 'order_date'].astype(str).map(month_map)

# Forward fill month values
df['order_date'] = df['order_date'].ffill()

# Handle day values under month headers (convert to string first)
day_mask = df['order_date'].astype(str).str.match(r'^\d{1,2}$', na=False)
df.loc[day_mask, 'order_date'] = (
    '2022-' + df.loc[day_mask, 'order_date'].shift().astype(str) + '-' + df.loc[day_mask, 'order_date'].astype(str).str.zfill(2)
)

# Correctly parse the dates with specified format to avoid warnings
df['order_date'] = pd.to_datetime(df['order_date'], format='%m/%d/%Y', errors='coerce')

# Format the dates as MM/DD/YY (optional if you want to display/save like this)
df['order_date'] = df['order_date'].dt.strftime('%m/%d/%y')

# Check the first 5 rows to verify
df['order_date'].head(10)




C:\Users\shrav\AppData\Local\Temp\ipykernel_11924\378665679.py:9: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce', dayfirst=True)


0    04/30/22
1    04/30/22
2    04/30/22
3    04/30/22
4    04/30/22
5    04/30/22
6    04/30/22
7    04/30/22
8    04/30/22
9    04/30/22
Name: order_date, dtype: object

In [68]:
#Handlin the missing values (Lets clean the data)
df['courier_status']= df['courier_status'].fillna('Pending')
df['currency']=df['currency'].fillna('Unknown')
df['order_amount']=df['order_amount'].fillna(0)
shipping_cols=['ship_city','ship_state', 'ship_postal_code', 'ship_country']
df[shipping_cols]=df[shipping_cols].fillna('Unknown')
df['promotion_ids']=df['promotion_ids'].fillna('None')
df['fulfilled_by']=df['fulfilled_by'].fillna('Seller')
print(df.isna().sum())



order_id              0
order_date            0
status                0
fulfilment            0
sales_channel         0
ship_service_level    0
style                 0
sku                   0
category              0
size                  0
asin                  0
courier_status        0
quantity              0
currency              0
order_amount          0
ship_city             0
ship_state            0
ship_postal_code      0
ship_country          0
promotion_ids         0
b2b                   0
fulfilled_by          0
total_order_sales     0
dtype: int64


In [69]:
#Total Sales
df['total_order_sales']= df['quantity']*df['order_amount']
df['total_order_sales'].sum()   #Total sum of sales
df.nlargest(5,'total_order_sales') #Top 5 Sales



,order_id,order_date,status,fulfilment,sales_channel,ship_service_level,style,sku,category,size,...,currency,order_amount,ship_city,ship_state,ship_postal_code,ship_country,promotion_ids,b2b,fulfilled_by,total_order_sales
42039,402-5239558-4281957,05/04/22,Shipped,Amazon,Amazon.in,Expedited,SET268,SET268-KR-NP-XS,Set,XS,...,INR,5584.0,GUNTUR,ANDHRA PRADESH,522007.0,IN,None,False,Seller,44672.0
11160,403-4106553-1691525,04/23/22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,J0013,J0013-SKD-XXXL,Set,3XL,...,INR,5495.0,JALANDHAR,PUNJAB,144022.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,True,Easy Ship,27475.0
89150,406-7195963-2374765,02/05/22,Shipped,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-M,Set,M,...,INR,3036.0,LUCKNOW,UTTAR PRADESH,226002.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,Seller,12144.0
39944,408-3363121-6123562,06/04/22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-XS,Western Dress,XS,...,INR,2860.0,KANPUR,UTTAR PRADESH,208001.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,11440.0
79899,405-6877720-7865168,07/05/22,Shipped - Returned to Seller,Merchant,Amazon.in,Standard,JNE2086,JNE2086-KR-389-M,kurta,M,...,INR,2796.0,MAHENDRAGARH,HARYANA,123029.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,11184.0


In [70]:
#Load the data into sql server
#Load the data into sql server
import sqlalchemy as sal
engine = sal.create_engine(
    r'mssql+pyodbc://DESKTOP-HI47A2M\SQLEXPRESS01/master?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
)
conn = engine.connect()
print("Connected successfully!")


Connected successfully!


In [71]:
df=df.drop(columns='index',errors='ignore')
df.to_sql('Amazon_Sales_Report', con=engine,index= False ,if_exists='append')

28

In [72]:
df.columns


Index(['order_id', 'order_date', 'status', 'fulfilment', 'sales_channel',
       'ship_service_level', 'style', 'sku', 'category', 'size', 'asin',
       'courier_status', 'quantity', 'currency', 'order_amount', 'ship_city',
       'ship_state', 'ship_postal_code', 'ship_country', 'promotion_ids',
       'b2b', 'fulfilled_by', 'total_order_sales'],
      dtype='object')